In [1]:
import numpy as np
import cv2 as cv
import face_recognition
import glob
from PIL import Image
import sys
from pylibfreenect2 import Freenect2, SyncMultiFrameListener
from pylibfreenect2 import FrameType, Registration, Frame
from pylibfreenect2 import createConsoleLogger, setGlobalLogger
from pylibfreenect2 import LoggerLevel

In [2]:
LEFT_CAMERA = "015001743347".encode('utf-8')
RIGHT_CAMERA = "009747143547".encode('utf-8')
# kinect basic 
try:
    from pylibfreenect2 import OpenGLPacketPipeline
    pipeline = OpenGLPacketPipeline()
except:
    try:
        from pylibfreenect2 import OpenCLPacketPipeline
        pipeline = OpenCLPacketPipeline()
    except:
        from pylibfreenect2 import CpuPacketPipeline
        pipeline = CpuPacketPipeline()
print("Packet pipeline:", type(pipeline).__name__)

# Create and set logger
logger = createConsoleLogger(LoggerLevel.Debug)
setGlobalLogger(logger)

fn = Freenect2()
num_devices = fn.enumerateDevices()
if num_devices == 0:
    print("No device connected!")
    sys.exit(1)

device = fn.openDevice(RIGHT_CAMERA, pipeline=pipeline)

listener = SyncMultiFrameListener(
    FrameType.Color | FrameType.Ir | FrameType.Depth)

# Register listeners
device.setColorFrameListener(listener)
device.setIrAndDepthFrameListener(listener)

device.start()

# NOTE: must be called after device.start()
registration = Registration(device.getIrCameraParams(),
                            device.getColorCameraParams())

undistorted = Frame(512, 424, 4)
registered = Frame(512, 424, 4)

# Optinal parameters for registration
# set True if you need
need_bigdepth = False
need_color_depth_map = False

bigdepth = Frame(1920, 1082, 4) if need_bigdepth else None
color_depth_map = np.zeros((424, 512),  np.int32).ravel() \
    if need_color_depth_map else None

# set video container
fourcc = cv.VideoWriter_fourcc(*'FMP4') 
videoRGB = cv.VideoWriter('videoRGB.mp4', fourcc, 12, (1920, 1080))
videoIR = cv.VideoWriter('videoIR.mp4', fourcc, 12, (512, 424), False)
videoDepth = cv.VideoWriter('videoDepth.mp4', fourcc, 12, (512, 424), False)

t1 = []
t2 = []
t3 = []
t4 = []
previousTimestamp = 0
fpsArr = []

RECORD = False
RGB_IMG_COUNTER = 0
D_IMG_COUNTER = 0

while True:
    frames = listener.waitForNewFrame()

    color = frames["color"]
    ir = frames["ir"]
    depth = frames["depth"]

    registration.apply(color, depth, undistorted, registered,
                       bigdepth=bigdepth,
                       color_depth_map=color_depth_map)
    # NOTE for visualization:
    # cv2.imshow without OpenGL backend seems to be quite slow to draw all
    # things below. Try commenting out some imshow if you don't have a fast
    # visualization backend.
    np_ir = ir.asarray()
    np_depth = depth.asarray()
    np_color = color.asarray()
    np_registered = registered.asarray(np.uint8)
    
    np_ir /= np_ir.max()
    np_ir = np.sqrt(np_ir)
    np_ir = np.array(Image.fromarray(256 * np_ir).convert('L'))
    
    np_depth /= np_depth.max()
    np_depth = np.sqrt(np_depth)
    np_depth = np.array(Image.fromarray(256 * np_depth).convert('L'))
    
    np_color = np_color[:,:,:3]
    
    cv.imshow("ir", np_ir)
    cv.imshow("depth", np_depth)
    cv.imshow("color", np_color)
    cv.imshow("registered", np_registered)
    
    
    if RECORD == True:
        videoIR.write(np_ir)
        videoRGB.write(np_color)
        videoDepth.write(np_depth)

    if need_bigdepth:
        cv.imshow("bigdepth", cv.resize(bigdepth.asarray(np.float32),
                                          (int(1920 / 3), int(1082 / 3))))
    if need_color_depth_map:
        cv.imshow("color_depth_map", color_depth_map.reshape(424, 512))

    listener.release(frames)

    key = cv.waitKey(delay=1)
    if key == ord('s'):
        cv.imwrite(f'calib_img_rgb{RGB_IMG_COUNTER}.jpg', np_color)
        RGB_IMG_COUNTER+=1
    if key == ord('d'):
        cv.imwrite(f'calib_img_d{D_IMG_COUNTER}.png', np_ir)
        D_IMG_COUNTER+=1
    elif key == ord('q'):
        break

device.stop()
device.close()

cv.destroyAllWindows()
videoRGB.release()
videoIR.release()
videoDepth.release()

Packet pipeline: OpenGLPacketPipeline


In [12]:
# calib img
CHECKERBOARD = (9,6)

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((1, CHECKERBOARD[0]* CHECKERBOARD[1],3), np.float32)
objp[0, :, :2] = np.mgrid[0:CHECKERBOARD[0],0:CHECKERBOARD[1]].T.reshape(-1, 2)

# rgb camera calib
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('calib_img_rgb*.jpg')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD, None)
    if ret == True:
        print(fname)
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
camera_file = {"ret": ret, "mtx": mtx, "dist": dist, "rvecs": rvecs, "tvecs": tvecs}

# depth camera calib   
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

ir_images = glob.glob('calib_img_d*.png')

for fname in ir_images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD, None)
    if ret == True:
        print(fname)
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
             
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
depth_camera_file = {"ret": ret, "mtx": mtx, "dist": dist, "rvecs": rvecs, "tvecs": tvecs}


calib_img_rgb8.jpg
calib_img_rgb9.jpg
calib_img_rgb2.jpg
calib_img_rgb3.jpg
calib_img_rgb1.jpg
calib_img_rgb0.jpg
calib_img_rgb4.jpg
calib_img_rgb5.jpg
calib_img_rgb7.jpg
calib_img_rgb6.jpg
calib_img_rgb11.jpg
calib_img_rgb10.jpg
calib_img_rgb12.jpg
calib_img_rgb13.jpg
calib_img_d12.png
calib_img_d13.png
calib_img_d11.png
calib_img_d10.png
calib_img_d14.png
calib_img_d3.png
calib_img_d2.png
calib_img_d0.png
calib_img_d1.png
calib_img_d5.png
calib_img_d4.png
calib_img_d6.png
calib_img_d7.png
calib_img_d9.png
calib_img_d8.png


In [38]:
# undistort kinect img
img = cv.imread('calib_img_9.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

True

In [41]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs, tvecs, mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 44.46178636775677


In [15]:
# live checkerboard cube render with kinect

#checkerboard axis or cube to corner
def drawHelper(f):
    x = int(f * scale_factor)
    if x > 100000:
        x = 100000
    if x < 0:
        x = 0
    return x

def draw(img, corners, imgpts):
    if drawType:
        return drawCube(img, corners, imgpts)
    else:
        return drawAxis(img, corners, imgpts)

def drawAxis(img, corners, imgpts):
    corner = tuple(map(drawHelper, corners[0].ravel()))
    img = cv.line(img, corner, tuple(map(drawHelper, imgpts[0].ravel())), (255,0,0), 5)
    img = cv.line(img, corner, tuple(map(drawHelper, imgpts[1].ravel())), (0,255,0), 5)
    img = cv.line(img, corner, tuple(map(drawHelper, imgpts[2].ravel())), (0,0,255), 5)
    return img

def drawCube(img, corners, imgpts):
    imgpts = np.array([[drawHelper(f) for f in p] for p in np.int32(imgpts).reshape(-1,2)])
    # draw ground floor in green
    img = cv.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    # draw top layer in red color
    img = cv.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img

        
CHECKERBOARD = (9,6)
process_this_frame = 0
screen_scale_factor = 1
scale_factor = 1
drawType = 1
saved_corners2=[[[]]]
saved_imgpts = [[[]]]
saved_ir_corners2 = [[[]]]
saved_ir_imgpts = [[[]]]

axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

if drawType:
    axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])

    
# kinect init
try:
    from pylibfreenect2 import OpenGLPacketPipeline
    pipeline = OpenGLPacketPipeline()
except:
    try:
        from pylibfreenect2 import OpenCLPacketPipeline
        pipeline = OpenCLPacketPipeline()
    except:
        from pylibfreenect2 import CpuPacketPipeline
        pipeline = CpuPacketPipeline()
print("Packet pipeline:", type(pipeline).__name__)

fn = Freenect2()
num_devices = fn.enumerateDevices()
if num_devices == 0:
    print("No device connected!")
    sys.exit(1)

device = fn.openDevice(LEFT_CAMERA, pipeline=pipeline)

listener = SyncMultiFrameListener(
    FrameType.Color | FrameType.Ir)

# Register listeners
device.setColorFrameListener(listener)
device.setIrAndDepthFrameListener(listener)

device.start()

while True:
    frames = listener.waitForNewFrame()
    color = frames["color"]
    ir = frames["ir"]
    
    np_color = color.asarray()
    np_color = np_color[:,:,:3]
    frame = np_color
    
    ir = ir.asarray()
    ir /= ir.max()
    ir = np.sqrt(ir)
    np_ir = np.array(Image.fromarray(256 * ir).convert('L'))
    ir_frame = np_ir

    frame = cv.resize(frame, (frame.shape[1]//screen_scale_factor, frame.shape[0]//screen_scale_factor), interpolation = cv.INTER_CUBIC)
    process_this_frame-=1
    
    if process_this_frame <= 0:
        process_this_frame = 5
        small_frame = cv.resize(frame, (0, 0), fx=1 / scale_factor, fy=1 / scale_factor)
        gray = cv.cvtColor(small_frame,cv.COLOR_BGR2GRAY)
        ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD,None)
        
        small_ir_frame = cv.resize(ir_frame, (0, 0), fx=1 / scale_factor, fy=1 / scale_factor)
        ir_gray = small_ir_frame
        ir_ret, ir_corners = cv.findChessboardCorners(ir_gray, CHECKERBOARD,None)

        if ret:
            corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            ret,rvecs, tvecs = cv.solvePnP(objp, corners2, camera_file["mtx"], camera_file["dist"])
            imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, camera_file["mtx"], camera_file["dist"])
            saved_corners2 = corners2
            saved_imgpts = imgpts
        
        if ir_ret:
            corners2 = cv.cornerSubPix(ir_gray,ir_corners, (11,11), (-1,-1), criteria)
            ret,rvecs, tvecs = cv.solvePnP(objp, corners2, depth_camera_file["mtx"], depth_camera_file["dist"])
            imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, depth_camera_file["mtx"], depth_camera_file["dist"])
            saved_ir_corners2 = corners2
            saved_ir_imgpts = imgpts
            
    listener.release(frames)
    if len(saved_corners2) > 1:
        frame = draw(frame,saved_corners2,saved_imgpts)
    if len(saved_ir_corners2) > 1:
        ir_frame = draw(ir_frame,saved_ir_corners2,saved_ir_imgpts)
    cv.imshow("Live", frame)
    cv.imshow("IR", ir_frame)
    if cv.waitKey(20) & 0xFF==ord('q'):
        break 

cv.destroyAllWindows()

device.stop()
device.close()


Packet pipeline: OpenGLPacketPipeline


In [73]:
for fname in glob.glob('*.jpg'):
    img = cv.imread(fname)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD,None)
    if ret == True:
        corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        ret,rvecs, tvecs = cv.solvePnP(objp, corners2, camera_file["mtx"], camera_file["dist"])
        # project 3D points to image plane
        imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, camera_file["mtx"], camera_file["dist"])
        img = draw(img,corners2,imgpts)
        cv.imshow('img',img)
        k = cv.waitKey(0) & 0xFF
        if k == ord('s'):
            cv.imwrite(fname+'.png', img)
cv.destroyAllWindows()

In [31]:
mtx

array([[1.33570580e+03, 0.00000000e+00, 6.00272754e+02],
       [0.00000000e+00, 1.32726396e+03, 4.45745390e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])